# Data Dictionary:

race_id: str
>Identifier for each race. The first race of the season will be 1. The last race will be 819. 

race: str
>The race number of each racing day.

date: time
>Date of the race.

place: str
>The order of horse complete the race track

horse_id: str
>Identifier for each horse

horse_no: str
>Number of the horse for that race

horse: str
>Name of the horse

jockey: str
>Name of the jockey

trainer: str
>Name of the trainer

actual_weight: float
>Weight that added on the horse

declared_horse_weight: float
>Weight of the horse

draw: str
>The stall where the horse started from.

lbw: float
>Distance between the horse and the winner when winner pass the rush line

running_position: array
>Place of the horse pass different check points

win_odds: float
>Ration of beting this horse if the horse win the 1st place

class: str
>The level/grade of the horse. If there is 'Group' in class, it is a international standard race.

going: str
>Condition of race track

track: str
>Field type of the race.Turf or mud (all weather track).

prize: int
>Prize for winner

location: str
>Location of the racecourse

distance_m: int
>Distance for this race

finish_time: time
>Time to finish the race

finish_time_s: float
>Time to finish the race in second

## Please use the follow dictionary for read_csv
```
types_dict = {'race_id': int, 'race': int, 'date':str, 'place':int, 'horse_id':str, 'horse_no':str,
    'horse':str, 'jockey':str, 'trainer':str, 'actual_weight':float, 'declared_horse_weight':float,
    'draw':int, 'lbw':float, 'running_position': str,'win_odds':float, 'class':str, 'going':str,
    'track':str, 'prize':int, 'location':str, 'distance_m':int, 'finish_time':str, 'finish_time_s':float}
parse_dates = ['date', 'finish_time']
raw=pd.read_csv('../data/race_2016_to_2020.csv',dtype=types_dict,parse_dates=parse_dates)
```

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
raw = pd.read_csv('../data/raw/merge_20_now.csv')
raw.head()

FileNotFoundError: [Errno 2] File ../data/raw/merge_20_now.csv does not exist: '../data/raw/merge_20_now.csv'

In [ ]:
raw.shape

In [ ]:
raw.info()

In [ ]:
#Drop samples with place has no digits
#Change places with DH to digit only
#reset index of dataframe
df_place=raw.copy()
df_place=df_place.drop_duplicates()
df_place=df_place[(df_place['place']!='WV')&(df_place['place']!='WV-A')]
df_place=df_place[(df_place['place']!='WX')&(df_place['place']!='WX-A')&(df_place['place']!='WXNR')]
df_place=df_place[(df_place['place']!='PU')&(df_place['place']!='UR')&(df_place['place']!='DNF')&(df_place['place']!='TNP')]
df_place=df_place[(df_place['place']!='FE')&(df_place['place']!='DISQ')]
df_place['place']=df_place['place'].str.extract(r'(\d+)')
df_place=df_place.reset_index(drop=True)
#Change horse no. to int
df_place['horse_no']=df_place['horse_no'].astype(int).astype(str)

In [ ]:
#split horse name into house name and horse id
df_horse = df_place.copy()
df_horse['horse_id']=df_horse['horse'].str.extract(r'\((.+)\)')
df_horse['horse']=df_horse['horse'].str.extract(r'^(.+)\(')

#change actual weight into float
df_horse['actual_weight']=df_horse['actual_weight'].astype(float)

#change declared horse weight into float
df_horse['declared_horse_weight']=df_horse['declared_horse_weight'].astype(float)

In [ ]:
#change lbw into float
df_lbw=df_horse.copy()
df_lbw.loc[df_lbw['lbw']=='-','lbw']='0'

def lbw_to_float(lbw_string):
    if lbw_string=='N':
        return 0.33
    elif lbw_string=='SH':
        return 0.1
    elif lbw_string=='HD':
        return 0.25
    elif lbw_string=='NOSE':
        return 0.02
    elif lbw_string=='ML':
        return 150
    nums = [int(i) for i in re.findall('\d+',lbw_string)]
    if len(nums)==1:
        return nums[0]
    elif len(nums)==2:
        return nums[0]/nums[1]
    elif len(nums)==3:
        return nums[0]+nums[1]/nums[2]
    else:
        return 999999

df_lbw['lbw']=df_lbw['lbw'].apply(lbw_to_float)

#change running position into array
df_lbw['running_position']=df_lbw['running_position'].str.findall(r'(\d+)')

In [ ]:
df_lbw[df_lbw['finish_time']=='---']

In [ ]:
#change finish_time into second with type float
df_time=df_lbw.copy()
def get_sec(finish_time):
    result=[float(i) for i in finish_time.split(':')]
    return result[0]*60+result[1]
df_time['finish_time_s']=df_time['finish_time'].apply(get_sec)
df_time['finish_time']=pd.to_datetime(df_time['finish_time'],format='%M:%S.%f')
df_time['win_odds']=df_time['win_odds'].astype(float)

In [ ]:
#split race and race_id
df_race=df_time.copy()
df_race['race_id']=df_race['race'].str.extract(r'\((\d+)')
df_race['race']=df_race['race'].str.extract(r'(\d+)\(')

In [ ]:
#extract number and class from original class data
def get_level(race_class):
    result=race_class.split('-')
    return result[0].strip()

def get_distance(race_class):
    result=race_class.split('-')
    return int(result[1].strip()[:-1])

df_race['distance_m']=df_race['class'].apply(get_distance)
df_race['class']=df_race['class'].apply(get_level)

In [ ]:
#extract digit from prize
#transform date to datetime type
df_prize=df_race.copy()
df_prize['prize']=df_prize['prize'].str.replace(r'[^0-9]', "").astype(int)
df_prize['date']=pd.to_datetime(df_prize['date'])

In [ ]:
#rename column turf to track 
#reorder the columns
df_final=df_prize.copy()
df_final['track']=df_final['turf']
df_final=df_final.drop(columns='turf',axis=1)
df_final=df_final[['race_id', 'race', 'date', 'place', 'horse_id', 'horse_no', 'horse', 'jockey', 'trainer', 'actual_weight',
       'declared_horse_weight', 'draw', 'lbw', 'running_position', 'win_odds', 'class', 'going', 'track',
       'prize', 'location', 'distance_m', 'finish_time', 'finish_time_s']]

In [ ]:
df_final

In [ ]:
df_final.to_csv('../data/race_s2021.csv',index=False)

In [ ]:
df_final.info(),df_final.shape

In [ ]:
types_dict = {'race_id': str, 'race': str, 'date':str, 'place':int, 'horse_id':str, 'horse_no':str,
              'horse':str, 'jockey':str, 'trainer':str, 'actual_weight':float, 'declared_horse_weight':float,
              'draw':int, 'lbw':float, 'running_position': str,'win_odds':float, 'class':str, 'going':str,
              'track':str, 'prize':int, 'location':str, 'distance_m':int, 'finish_time':str, 'finish_time_s':float}
parse_dates = ['date', 'finish_time']

df=pd.read_csv('../data/race_s2021.csv',dtype=types_dict,parse_dates=parse_dates)

In [ ]:
df.info()

In [ ]:
df